In [1]:
from requests import exceptions
import requests
import cv2
import os
import gevent

In [2]:
searchengine = 'r2d2'
output = 'download/r2d2'

In [3]:
API_KEY = "034200e7647f4157aadee9ebd49fcbe6"
MAX_RESULTS = 50
GROUP_SIZE = 250

In [4]:
URL = "https://api.bing.microsoft.com/v7.0/images/search"
       


EXCEPTIONS = {IOError, FileNotFoundError, exceptions.RequestException, exceptions.HTTPError, exceptions.ConnectionError,
              exceptions.Timeout}

In [5]:
term = searchengine
headers = {"Ocp-Apim-Subscription-Key": API_KEY}
params = {"q": term, "offset": 0, "count": GROUP_SIZE}

In [6]:
print("[INFO] searching Bing API for '{}'".format(term))
search = requests.get(URL, headers=headers, params=params)
search.raise_for_status()

[INFO] searching Bing API for 'r2d2'


In [7]:
total = 0
results = search.json()
estNumResults = min(results["totalEstimatedMatches"], MAX_RESULTS)
print("[INFO] {} total results for '{}'".format(estNumResults, term))

[INFO] 50 total results for 'r2d2'


In [8]:
def grab_page(url, ext, total):

    try:
        print("[INFO] fetching: {}".format(url))
        r = requests.get(url, timeout=30)
        p = os.path.sep.join([output, "{}{}".format(
            str(total).zfill(8), ext)])

        f = open(p, "wb")
        f.write(r.content)
        f.close()

        image = cv2.imread(p)

        if image is None:
            print("[INFO] deleting: {}".format(p))
            os.remove(p)
            return

    except Exception as e:
        if type(e) in EXCEPTIONS:
            print("[INFO] skipping: {}".format(url))
            return

In [9]:
for offset in range(0, estNumResults, GROUP_SIZE):
    print("[INFO] making request for group {}-{} of {}...".format(
        offset, offset + GROUP_SIZE, estNumResults))
    params["offset"] = offset
    search = requests.get(URL, headers=headers, params=params)
    search.raise_for_status()
    results = search.json()
    print("[INFO] saving images for group {}-{} of {}...".format(
        offset, offset + GROUP_SIZE, estNumResults))
    jobs = []
    for v in results["value"]:
        total += 1
        ext = v["contentUrl"][v["contentUrl"].rfind("."):]
        url = v["contentUrl"]
        
        jobs.append(gevent.spawn(grab_page, url, ext, total))

    gevent.joinall(jobs, timeout=10)
    print(total)

[INFO] making request for group 0-250 of 50...
[INFO] saving images for group 0-250 of 50...
[INFO] fetching: https://img1.cgtrader.com/items/797358/51b7673c0f/r2d2-star-wars-3d-model-obj-ztl.png
[INFO] fetching: https://vignette.wikia.nocookie.net/disney/images/0/05/ForceAwakensR2-D2Fathead.png/revision/latest/scale-to-width-down/2000?cb=20160207021436
[INFO] skipping: https://vignette.wikia.nocookie.net/disney/images/0/05/ForceAwakensR2-D2Fathead.png/revision/latest/scale-to-width-down/2000?cb=20160207021436
[INFO] fetching: https://img-new.cgtrader.com/items/27610/r2d2_star_wars_droid_robot_3d_model_3ds_fbx_c4d_lwo_lw_lws_obj_max__cf59dc7f-6bc6-4646-8cf7-334c1504921a.jpg
[INFO] fetching: https://fthmb.tqn.com/eeLON8lcmgEdkEFpKzTsHLmFpf4=/793x800/filters:fill(auto,1)/R2D2_MCH-IA-324_R_8x10-56a839a65f9b58b7d0f1a855.jpg
[INFO] fetching: https://3riverscomicon.com/wp-content/uploads/2016/03/r2d2.png
[INFO] fetching: https://content.instructables.com/ORIG/F3X/W92I/H2WESV25/F3XW92IH2WESV2